In [6]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [28]:
dataset=pd.read_csv('./datasetFolder/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1")

In [29]:
dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [31]:
col_names=['target','id','date','flag','user','text']
dataset.columns=col_names
dataset.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [32]:
dataset.shape

(1599999, 6)

In [33]:
dataset.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [34]:
dataset['target'].value_counts()

target
4    800000
0    799999
Name: count, dtype: int64

In [35]:
dataset['target']=dataset['target'].map({4:1,0:0})

In [36]:
dataset['target'].value_counts()

target
1    800000
0    799999
Name: count, dtype: int64

In [79]:
    
stremmer = PorterStemmer() 

def stemming(content):
    if not isinstance(content, str):  # Skip non-string values
        return ""
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)  # Remove non-alphabetic chars
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stremmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    return ' '.join(stemmed_content)


In [80]:
content = "Running cats are playing outside."
result = stemming(content)
print(result)


run cat play outsid


In [81]:
dataset['text'] = dataset['text'].apply(stemming)

# Drop rows with NaN or empty strings
dataset = dataset[dataset['text'].str.strip() != ""]


In [82]:
dataset.to_csv('processed_dataset2.csv', index=False)
dataset = pd.read_csv('datasetFolder/processed_dataset2.csv', encoding="ISO-8859-1")

# Check and clean after reloading
dataset = dataset.dropna(subset=['text'])
dataset['text'] = dataset['text'].astype(str)


In [83]:
dataset.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset updat facebook text might cri result sch...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan dive mani time ball manag save rest g...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole bodi feel itchi like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass behav mad see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,kwesidei whole crew


In [84]:
dataset.isnull().sum()


target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [86]:
x = dataset['text']
y = dataset['target']

# Verify that x contains only strings
print(x.apply(type).value_counts())  # Should only show <class 'str'>

# Check for empty or invalid rows
print(x[x.str.strip() == ""])  # Should return an empty Series


text
<class 'str'>    1599494
Name: count, dtype: int64
Series([], Name: text, dtype: object)


In [87]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)


In [88]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9433454 stored elements and shape (1279595, 457000)>
  Coords	Values
  (0, 165532)	0.369687489493267
  (0, 331685)	0.3534319035195359
  (0, 294661)	0.5370176669489918
  (0, 272262)	0.4325275987219066
  (0, 14903)	0.23477283329695414
  (0, 157700)	0.2589900553053751
  (0, 269960)	0.31541337595606417
  (0, 232106)	0.2031482751142228
  (1, 20429)	0.5649526542981816
  (1, 431846)	0.38621160950712147
  (1, 246499)	0.4101517370575121
  (1, 298651)	0.3873262689205645
  (1, 286573)	0.4619772776621573
  (2, 167965)	0.6369855509346276
  (2, 130822)	0.31257196828312855
  (2, 403789)	0.33704316530522777
  (2, 147035)	0.2750009974255575
  (2, 354674)	0.2036230307734393
  (2, 412496)	0.305042963577446
  (2, 214156)	0.28013449031826143
  (2, 307436)	0.30714108765314585
  (3, 382528)	0.34700141155019143
  (3, 264813)	0.14625166592813918
  (3, 422490)	0.19736087929476312
  (3, 25665)	0.23122445487666668
  :	:
  (1279592, 127051)	0.2701827758

In [89]:
model=LogisticRegression()
model.fit(x_train, y_train)

D:\SentimentAnalyzer_Project\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [90]:
x_pred=model.predict(x_test)
print(accuracy_score(y_test,x_pred))

0.7783237834441495


In [1]:
def predict_sentiment(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [stremmer.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    text = [text]
    text = vectorizer.transform([text])
    sentiment = model.predict(text)
    if sentiment ==0:
        return "Negative sentence"
    else:
        return "Positive sentence"

In [2]:
print(predict_sentiment("I want to murder someone"))
print(predict_sentiment("I want to show mercy"))

NameError: name 're' is not defined